In [1]:
import glob
import os
import pandas as pd
import numpy as np
import sqlite3
import datetime


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.layers import Dropout
import theano

Using Theano backend.


In [2]:
#load files to DB

con = sqlite3.connect("SquareBrawlTrainingRawSQLite.db")

folders = glob.glob('training_data\\TrainOn\*')

#trains = pd.DataFrame()

for folder in folders:
    print(folder)
    trains = pd.DataFrame()
    folder = folder + '\\*'
    files = glob.glob(folder)
    sfiles = sorted(files, key=os.path.getctime)
    for sfile in sfiles:
        train = pd.read_csv(sfile)
        trains = pd.concat([trains, train])
    
    trains = trains.reset_index().drop('index',axis=1)
    trains.to_sql(name='SquareBrawlTrainingRaw', con=con, if_exists='append')


training_data\TrainOn\20160616_2  - close enough


c:\program files\anaconda3\envs\sb\lib\site-packages\pandas\core\generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


training_data\TrainOn\20160617
training_data\TrainOn\20160619
training_data\TrainOn\20160620
training_data\TrainOn\20160621
training_data\TrainOn\20170622
training_data\TrainOn\20170623


In [10]:
#connect to DB

con = sqlite3.connect("SquareBrawlTrainingRawSQLite.db")

SQL_STATEMENT = '''SELECT
episode,
    p1_x,
    " p1_y",
    p2_x,
    " p2_y",
    right_of, 
    left_of, 
    above, 
    below, 
    x_clear, 
    y_clear, 
    cd1,
    cd2, 
    aim_down, 
    aim_left, 
    aim_up, 
    aim_right,
    action,
    p1_score,
    p2_score
FROM
SquareBrawlTraining
'''

training_chunks = pd.read_sql(SQL_STATEMENT, con, chunksize=1000)

In [2]:
def score_delta_index(chunk, column_name):
    rows = chunk.shape[0]
    prev_score = 0
    prev_row = 0
    kills_index = []
    
    for row in range(rows):
        if chunk.loc[row, column_name] != prev_score:
            kills_index.append([prev_row, row])
            prev_score = chunk.loc[row, column_name]
            prev_row = row + 1
    del kills_index[0] # delete [0,0]
    
    return kills_index

In [3]:
def td_lambda_adjustments(chunk, adjustment_column, kills_index, reward, decay, td_lookback):
    # TD lamnda adjustments for kills
    for kill in kills_index:
        killpoint = kill[1]
        exponent = td_lookback - 1
        if killpoint - td_lookback < 0:
            continue
        else:
            for row in range(killpoint - td_lookback, killpoint):
                chunk.loc[row, adjustment_column] = reward * decay ** exponent
                exponent -= 1
        
    return chunk

In [4]:
#connect to DB

con = sqlite3.connect("SquareBrawlTrainingRawSQLite.db")

SQL_STATEMENT = '''SELECT
episode,
    p1_x,
    " p1_y",
    p2_x,
    " p2_y",
    right_of, 
    left_of, 
    above, 
    below, 
    x_clear, 
    y_clear, 
    cd1,
    cd2, 
    aim_down, 
    aim_left, 
    aim_up, 
    aim_right,
    action,
    p1_score,
    p2_score
FROM
SquareBrawlTraining
'''


In [5]:
#-------TD LAMBDA REWARDS ADJUSTED---------#
td_lookback = 25
decay = 0.9
reward_kill = 1
reward_death = -1


# load database with all raw training_data
chunksizes = 50000
training_chunks_raw = pd.read_sql(SQL_STATEMENT, con, chunksize=chunksizes)

#create new database for adjusted rewards
td_lambda_storage = sqlite3.connect("SquareBrawlSQLite_AdjustedRewards.db")

total_kills = []
total_deaths = []
episode = 0

for chunk in training_chunks_raw:
    chunk['reward_kill_adjusted'] = 0
    chunk['reward_death_adjusted'] = 0
    
    chunk = chunk.reset_index().drop('index',axis=1)
    
    # grab kills
    kills_index = score_delta_index(chunk,'p1_score')
    total_kills.extend(kills_index)
    chunk = td_lambda_adjustments(chunk, 'reward_kill_adjusted', kills_index, reward_kill, decay, td_lookback)
    
    # grab deaths
    deaths_index = score_delta_index(chunk, 'p2_score')
    total_deaths.extend(deaths_index)
    chunk = td_lambda_adjustments(chunk, 'reward_death_adjusted', deaths_index, reward_death, decay, td_lookback)
    
    chunk['adjusted_rewards_total'] = chunk['reward_kill_adjusted'] + chunk['reward_death_adjusted']

    chunk.to_sql(name='TD25_decay9_pos_neg_reward_1', con=td_lambda_storage, if_exists='append')
    
    print('Saving', episode)
    
    episode += 1
    
    

c:\program files\anaconda3\envs\sb\lib\site-packages\pandas\core\generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


Saving 0
Saving 1
Saving 2
Saving 3
Saving 4
Saving 5
Saving 6
Saving 7
Saving 8
Saving 9
Saving 10
Saving 11
Saving 12
Saving 13
Saving 14
Saving 15
Saving 16
Saving 17
Saving 18
Saving 19
Saving 20
Saving 21
Saving 22
Saving 23
Saving 24
Saving 25
Saving 26
Saving 27
Saving 28
Saving 29
Saving 30
Saving 31
Saving 32
Saving 33
Saving 34
Saving 35
Saving 36
Saving 37
Saving 38
Saving 39
Saving 40
Saving 41
Saving 42
Saving 43
Saving 44
Saving 45
Saving 46
Saving 47
Saving 48
Saving 49
Saving 50
Saving 51
Saving 52
Saving 53
Saving 54
Saving 55


In [10]:
print(len(total_kills), len(total_deaths))


6205 21786


In [9]:
def keras_model(inputs, dropouts=0.2, opti=1e-6, clipping=0.1, clipval=10):
    keras_model = Sequential()
    keras_model.add(Dense(128, input_dim=inputs, activation='relu'))
    keras_model.add(Dropout(dropouts))
    keras_model.add(Dense(128, activation='relu'))
    keras_model.add(Dropout(dropouts))
    keras_model.add(Dense(1))
    adam = Adam(lr=opti, clipnorm=clipping, clipvalue=clipval)
    keras_model.compile(loss='mse', optimizer='adam')
    return keras_model

In [10]:
#connect to DB

con = sqlite3.connect("SquareBrawlSQLite_AdjustedRewards.db")

SQL_STATEMENT = '''SELECT
    p1_x,
    " p1_y",
    p2_x,
    " p2_y",
    right_of, 
    left_of, 
    above, 
    below, 
    x_clear, 
    y_clear, 
    cd1,
    cd2, 
    aim_down, 
    aim_left, 
    aim_up, 
    aim_right,
    action,
    reward_kill_adjusted
FROM
TD25_decay9_pos_neg_reward_1
'''

In [11]:
chunksizes = 10
training_chunks_clean = pd.read_sql(SQL_STATEMENT, con, chunksize=chunksizes)
#grab out input shape size
for chunk in training_chunks_clean:
    input_shape = chunk.shape[1] - 1 # minus 1 for y target which is to be dropped
    print('input shape', input_shape)
    break

input shape 17


In [12]:
Qas = keras_model(inputs=input_shape, dropouts=0.2, opti=1e-6, clipping=1, clipval=10)
save_location = 'model_weights//total_rewards//'

In [13]:
chunksizes = 250000
training_chunks_clean = pd.read_sql(SQL_STATEMENT, con, chunksize=chunksizes)

In [ ]:
episode = 1

#grab out input shape size
for chunk in training_chunks_clean:
    y = np.array(chunk['reward_kill_adjusted'])
    drop_cols = ['reward_kill_adjusted']
    for drop in drop_cols:
        chunk.drop(drop, axis=1, inplace=True)
    training_set = np.array(chunk)
    Qas.fit(training_set, y, epochs=50, verbose=1)
    
    name =  save_location + str(episode*chunksizes) + '_model_backup_' + datetime.datetime.today().strftime('%Y%m%d_%H%M') + '.keras'
    Qas.save_weights(name)
    print('Saving ', name)
    
    episode += 1
    
    print('----------------------| ', episode*chunksizes/2735000*100, '% done |---------------------------------')

Epoch 1/50
250000/250000 [==============================] - 14s - loss: 0.0439    
Epoch 2/50
250000/250000 [==============================] - 28s - loss: 0.0092    
Epoch 3/50
250000/250000 [==============================] - 39s - loss: 0.0091    
Epoch 4/50
250000/250000 [==============================] - 44s - loss: 0.0091    
Epoch 5/50
250000/250000 [==============================] - 46s - loss: 0.0091    
Epoch 6/50
250000/250000 [==============================] - 46s - loss: 0.0091    
Epoch 7/50
250000/250000 [==============================] - 47s - loss: 0.0091    
Epoch 8/50
250000/250000 [==============================] - 48s - loss: 0.0091    
Epoch 9/50
250000/250000 [==============================] - 49s - loss: 0.0091    
Epoch 10/50
250000/250000 [==============================] - 48s - loss: 0.0091    
Epoch 11/50
250000/250000 [==============================] - 52s - loss: 0.0091    
Epoch 12/50
250000/250000 [==============================] - 56s - loss: 0.0091    
E

In [ ]:

    
    training_set = np.array(chunk)
   

    

In [ ]:
'''drop_cols = ['episode', 'reward', 'p1_score', 'p2_score',
             'p1_x_delta', 'p2_x_delta', 'p1_y_delta', 'p2_y_delta',
            'p1h', 'p2h']'''

In [13]:
[(10,2)]

[(10, 2)]

In [12]:
len(total_kills)

2407

In [9]:
episode

22

In [92]:
chunk

,episode,p1_x,p1_y,p1_x,p1_y,right_of,left_of,above,below,x_clear,...,cd1,cd2,aim_down,aim_left,aim_up,aim_right,action,p1_score,p2_score,reward_adjusted
0,7001,-10.2,-7.1,-10.2,-7.1,1,0,0,1,0,...,2,0,1,0,0,0,1,19,52,0.000000
1,7002,-11.4,-7.1,-11.4,-7.1,1,0,0,1,0,...,2,0,0,0,1,0,3,19,52,0.000000
2,7003,-12.1,-7.1,-12.1,-7.1,1,0,0,1,0,...,2,0,0,1,0,0,2,19,52,0.000000
3,7004,-12.0,-7.1,-12.0,-7.1,1,0,0,1,0,...,1,0,0,1,0,0,5,19,52,0.000000
4,7005,-11.5,-7.1,-11.5,-7.1,1,0,0,1,0,...,1,0,0,0,0,1,0,19,52,0.000000
5,7006,-10.4,-7.1,-10.4,-7.1,1,0,0,0,0,...,1,0,0,0,0,1,0,19,52,0.000000
6,7007,-9.3,-7.0,-9.3,-7.0,1,0,1,0,0,...,1,0,0,0,1,0,3,19,52,0.000000
7,7008,-8.1,-7.2,-8.1,-7.2,1,0,0,1,0,...,1,0,0,0,0,1,0,19,52,0.000000
8,7009,-6.6,-5.1,-6.6,-5.1,1,0,1,0,0,...,1,0,1,0,0,0,1,19,52,0.000000
9,7010,-8.2,-3.4,-8.2,-3.4,1,0,1,0,0,...,0,0,0,0,0,1,0,19,52,0.000000


In [26]:
drop_cols = ['episode', 'reward', 'p1_score', 'p2_score',
             'p1_x_delta', 'p2_x_delta', 'p1_y_delta', 'p2_y_delta',
            'p1h', 'p2h']

for item in drop_cols:
    trains.drop(item,inplace=True,axis=1)

MemoryError: 

In [22]:
trains = trains.reset_index().drop('index',axis=1)

MemoryError: 

In [24]:
rows = trains.shape[0]
prev_score = 0
prev_row = 0
kills = []
for row in range(rows):
    if trains.ix[row, 'p1_score'] != prev_score:
        kills.append([prev_row, row])
        prev_score = trains.ix[row, 'p1_score']
        prev_row = row + 1


MemoryError: 

In [18]:
trains['reward_adjusted'] = 0

In [1]:
len(kills)

NameError: name 'kills' is not defined

In [18]:
td_lookback = 25
decay = 0.9
reward = 1

for kill in kills:
    killpoint = kill[1]
    exponent = td_lookback - 1
    for row in range(killpoint - td_lookback, killpoint):
        trains.ix[row, 'reward_adjusted'] = reward * decay ** exponent
        exponent -= 1

In [2]:
#trains.to_csv('training_data_20170620.csv', index=False)
trains = pd.read_csv('training_data_20170620.csv')

In [3]:
trains = trains.drop('Unnamed: 0',axis=1)

In [4]:
trains_backup = trains.copy()

In [5]:
trains.head()

,episode,p1h,p1_x,p1_y,p2h,p2_x,p2_y,right_of,left_of,above,...,aim_right,p1_x_delta,p2_x_delta,p1_y_delta,p2_y_delta,action,reward,p1_score,p2_score,reward_adjusted
0,1,100.0,-3.0,-2.0,100.0,0.2,-6.9,1,0,1,...,0,0.0,0.0,0.0,0.0,1,0.0,0,0,0.0
1,2,100.0,-3.0,-2.0,100.0,0.2,-6.9,1,0,1,...,0,0.0,0.0,0.0,0.0,5,0.0,0,0,0.0
2,3,100.0,-6.2,-2.1,100.0,1.4,-7.0,1,0,1,...,0,-3.2,1.2,-0.1,-0.1,6,0.0,0,0,0.0
3,4,100.0,-8.9,-2.6,100.0,3.1,-7.0,1,0,1,...,1,-2.7,1.7,-0.5,0.0,0,0.0,0,0,0.0
4,5,100.0,-10.6,-3.2,100.0,4.7,-7.1,1,0,1,...,1,-1.7,1.6,-0.6,-0.1,4,0.0,0,0,0.0


In [6]:
drop_cols = ['episode', 'reward', 'p1_score', 'p2_score', 'reward_adjusted',
             'p1_x_delta', 'p2_x_delta', 'p1_y_delta', 'p2_y_delta',
            'p1h', 'p2h']
y = trains_backup['reward_adjusted']
for item in drop_cols:
    trains_backup.drop(item,inplace=True,axis=1)

In [7]:
trains_backup.head(2)

,p1_x,p1_y,p2_x,p2_y,right_of,left_of,above,below,x_clear,y_clear,cd1,cd2,aim_down,aim_left,aim_up,aim_right,action
0,-3.0,-2.0,0.2,-6.9,1,0,1,0,0,0,0,0,1,0,0,0,1
1,-3.0,-2.0,0.2,-6.9,1,0,1,0,0,0,0,0,1,0,0,0,5


Using Theano backend.


In [9]:
input_shape = trains_backup.shape[1]


Qas = keras_model(inputs=input_shape, dropouts=0.2, opti=1e-6, clipping=0.05, clipval=10)

In [10]:
training_set = np.array(trains_backup)
Y = np.array(y)

In [13]:
Qas.fit(training_set, Y, epochs=150, verbose=1)

Epoch 1/150
995000/995000 [==============================] - 307s - loss: 0.0110   
Epoch 2/150
995000/995000 [==============================] - 311s - loss: 0.0110   
Epoch 3/150
995000/995000 [==============================] - 308s - loss: 0.0110   
Epoch 4/150
995000/995000 [==============================] - 310s - loss: 0.0110   
Epoch 5/150
995000/995000 [==============================] - 314s - loss: 0.0110   
Epoch 6/150
995000/995000 [==============================] - 317s - loss: 0.0110   
Epoch 7/150
995000/995000 [==============================] - 322s - loss: 0.0110   
Epoch 8/150
995000/995000 [==============================] - 325s - loss: 0.0110   
Epoch 9/150
995000/995000 [==============================] - 326s - loss: 0.0110   
Epoch 10/150
995000/995000 [==============================] - 327s - loss: 0.0110   
Epoch 11/150
995000/995000 [==============================] - 325s - loss: 0.0110   
Epoch 12/150
995000/995000 [==============================] - 327s - loss:

KeyboardInterrupt: 

In [14]:

Qas.save_weights("SB-Qas-v2-20170620.keras")

In [32]:
datapoint = env.create_predictions(datapoint)
action_rewards = Qas.predict(predictions

SyntaxError: unexpected EOF while parsing (<ipython-input-32-6fec285e563b>, line 2)

In [ ]:
#action = np.where(advantage == np.amax(advantage))[0][0]
        #action = np.where(action_rewards == np.amax(action_rewards))[0][0] # greedy action

In [ ]:
bin(1000)